<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import numpy as np
import gensim
import os
import re
from ast import literal_eval

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
import tqdm

In [108]:
df = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')
print(df.shape)
df.head()

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(28332, 24)


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [7]:
# Notice anything odd here? 
df['brand'].value_counts()

Amazon          16153
Amazonbasics    12169
AmazonBasics       10
Name: brand, dtype: int64

In [8]:
# Much cleaner
df['brand'] = df['brand'].apply(lambda x: x.lower())
df['brand'].value_counts()

amazon          16153
amazonbasics    12179
Name: brand, dtype: int64

In [10]:
#stopwords

STOPWORDS = set(STOPWORDS).union(set(['amazon', 'amazonbasics']))

#tokenizer function

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [11]:
#Tokenize reviews.text column 

df['tokens'] = df['reviews.text'].apply(lambda x: tokenize(x))

In [12]:
#Dictionary Representation of all the words in our corpus

id2word = corpora.Dictionary(df['tokens'])

In [13]:
#list out 20 id2word items

list(id2word.items())[:20]

[(0, 'aluminum'),
 (1, 'backup'),
 (2, 'bad'),
 (3, 'battery'),
 (4, 'item'),
 (5, 'missing'),
 (6, 'order'),
 (7, 'pcs'),
 (8, 'quality'),
 (9, 'spring'),
 (10, 'work'),
 (11, 'bulk'),
 (12, 'expensive'),
 (13, 'like'),
 (14, 'products'),
 (15, 'way'),
 (16, 'duracell'),
 (17, 'happy'),
 (18, 'price'),
 (19, 'batteries')]

In [14]:
#Make a corpus with bag of words
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [70]:
# Make a LDA Topic Model

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=42,
                   num_topics = 20, # depends on the corpus
                   passes=10,
                   workers=12
                  )

#sklearn does all the computations right away, but gensim does not do this, it uses the print_topics() method

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [71]:
lda.print_topics()

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.049*"read" + 0.041*"kindle" + 0.038*"books" + 0.031*"love" + 0.015*"book" + 0.013*"easy" + 0.012*"new" + 0.010*"old" + 0.010*"reading" + 0.010*"purchased"'),
 (1,
  '0.040*"good" + 0.034*"far" + 0.028*"great" + 0.025*"battery" + 0.025*"life" + 0.022*"kindle" + 0.016*"long" + 0.015*"tablet" + 0.013*"love" + 0.010*"wife"'),
 (2,
  '0.071*"kids" + 0.034*"great" + 0.034*"love" + 0.026*"easy" + 0.024*"tablet" + 0.023*"loves" + 0.023*"use" + 0.021*"case" + 0.015*"durable" + 0.014*"bought"'),
 (3,
  '0.017*"batteries" + 0.015*"happy" + 0.013*"bought" + 0.011*"box" + 0.011*"cheap" + 0.011*"purchase" + 0.010*"tablet" + 0.010*"great" + 0.009*"price" + 0.009*"better"'),
 (4,
  '0.125*"batteries" + 0.031*"long" + 0.028*"work" + 0.027*"price" + 0.026*"good" + 0.021*"brand" + 0.018*"great" + 0.017*"battery" + 0.015*"brands" + 0.014*"buy"'),
 (5,
  '0.047*"use" + 0.043*"tablet" + 0.037*"easy" + 0.030*"perfect" + 0.023*"size" + 0.019*"games" + 0.019*"books" + 0.016*"great" + 0.015*"screen" +

In [72]:
#From lda.print_topics, clean them when regex
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [73]:
#Join words to form the topics
topics = [' '.join(t[0:5]) for t in words]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [74]:
#Print Topics - neatly done from lecture

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
read kindle books love book

------ Topic 1 ------
good far great battery life

------ Topic 2 ------
kids great love easy tablet

------ Topic 3 ------
batteries happy bought box cheap

------ Topic 4 ------
batteries long work price good

------ Topic 5 ------
use tablet easy perfect size

------ Topic 6 ------
great price tablet product love

------ Topic 7 ------
bought kindle new use like

------ Topic 8 ------
kindle tablet light screen best

------ Topic 9 ------
like screen kindle tablet keyboard

------ Topic 10 ------
works great tablet buy best

------ Topic 11 ------
good time set tablet like

------ Topic 12 ------
like tablet pretty good little

------ Topic 13 ------
store google play tablet use

------ Topic 14 ------
tablet great good price old

------ Topic 15 ------
price tablet good bought black

------ Topic 16 ------
loves old year tablet bought

------ Topic 17 ------
tablet apps good use like

------ Topic 18 ------
easy use tablet love app

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [75]:
#Vis import
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [76]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.263260  0.108565       1        1  17.883375
16     0.195382  0.071177       2        1   7.311549
8     -0.049274 -0.110291       3        1   6.833475
19    -0.110974  0.084894       4        1   5.311231
6      0.033702  0.125543       5        1   5.178595
5      0.101532 -0.068165       6        1   4.831956
17     0.027837 -0.030524       7        1   4.682528
2      0.169326  0.105744       8        1   4.667338
18     0.151589  0.054063       9        1   4.437432
13    -0.033730 -0.080685      10        1   4.347311
3     -0.112627  0.003001      11        1   4.188075
0      0.057731 -0.148889      12        1   4.028246
7     -0.004929 -0.102591      13        1   3.758989
15    -0.024273  0.072594      14        1   3.460054
9     -0.009548 -0.108574      15        1   3.431746
10    -0.024943 -0.005344      16        1   3.293309
12    -0.013224 -0.035891      17        1   3.183759
1     -0.081423  0.017909      18        1   3.100518
14     0.021395  0.093245      19        1   3.059091
11    -0.030289 -0.045781      20        1   3.011423, topic_info=           Term         Freq        Total Category  logprob  loglift
19    batteries  7373.000000  7373.000000  Default  30.0000  30.0000
22        great  8608.000000  8608.000000  Default  29.0000  29.0000
33         good  5478.000000  5478.000000  Default  28.0000  28.0000
2614      loves  2213.000000  2213.000000  Default  27.0000  27.0000
18        price  4579.000000  4579.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
28          buy    52.511568  2203.920032  Topic20  -5.1963  -0.2342
203         far    43.396147  1142.604486  Topic20  -5.3870   0.2321
1143    reading    44.118919  1267.510240  Topic20  -5.3705   0.1448
18        price    53.770141  4579.898261  Topic20  -5.1726  -0.9420
233        best    42.102647  1397.184378  Topic20  -5.4172   0.0007

[1627 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
56        1  0.976153    aa
56        2  0.002117    aa
56       11  0.008470    aa
56       13  0.002117    aa
56       18  0.002117    aa
...     ...       ...   ...
6328      8  0.038003   äôt
6328      9  0.038003   äôt
6328     15  0.456032   äôt
6328     17  0.095007   äôt
8920     17  0.919947  äôve

[6425 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 17, 9, 20, 7, 6, 18, 3, 19, 14, 4, 1, 8, 16, 10, 11, 13, 2, 15, 12])

In [64]:
distro = [lda[doc] for doc in corpus]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [65]:
distro[0]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(4, 0.17438598), (8, 0.19476548), (15, 0.27583966), (22, 0.28271168)]

In [43]:
def update(doc):
    d_dist = {k:0 for k in range(0,20)}
    for t in doc:
        d_dist[t[0]] = t[1]
    return d_dist

new_distro = [update(doc) for doc in distro] 

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [77]:
# Update the distribution to a complete distribution

distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [78]:
distro[0]

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(3, 0.19327666), (4, 0.25733048), (17, 0.11849391), (19, 0.3641194)]

In [79]:
len(new_distro)

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


28332

In [80]:
# Make a new dataframe called tf for topicframe, for the distribution

mf = pd.DataFrame.from_records(new_distro, index=df.primaryCategories)
mf.columns = topics

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [84]:
df['primaryCategories'].value_counts()

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Electronics                    13995
Health & Beauty                12071
Toys & Games,Electronics        1676
Office Supplies,Electronics      386
Electronics,Media                185
Office Supplies                    9
Animals & Pet Supplies             6
Electronics,Furniture              2
Home & Garden                      2
Name: primaryCategories, dtype: int64

In [85]:
list(df)

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['id',
 'dateAdded',
 'dateUpdated',
 'name',
 'asins',
 'brand',
 'categories',
 'primaryCategories',
 'imageURLs',
 'keys',
 'manufacturer',
 'manufacturerNumber',
 'reviews.date',
 'reviews.dateSeen',
 'reviews.didPurchase',
 'reviews.doRecommend',
 'reviews.id',
 'reviews.numHelpful',
 'reviews.rating',
 'reviews.sourceURLs',
 'reviews.text',
 'reviews.title',
 'reviews.username',
 'sourceURLs',
 'tokens']

In [109]:
print(mf.shape)
mf.tail()

(28332, 20)


C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,read kindle books love book,good far great battery life,kids great love easy tablet,batteries happy bought box cheap,batteries long work price good,use tablet easy perfect size,great price tablet product love,bought kindle new use like,kindle tablet light screen best,like screen kindle tablet keyboard,works great tablet buy best,good time set tablet like,like tablet pretty good little,store google play tablet use,tablet great good price old,price tablet good bought black,loves old year tablet bought,tablet apps good use like,easy use tablet love apps,good great value quality product
primaryCategories,,,,,,,,,,,,,,,,,,,,
Electronics,0.162112,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,0.756060,NaN
Electronics,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,0.881249,NaN
Electronics,0.394553,0.129259,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.242452,0.0,0.141403,0.0,0.0,0.0,0.069583,NaN,NaN,NaN,NaN
Electronics,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.954747,NaN,NaN,NaN,NaN
Electronics,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.096497,0.000000,0.0,0.000000,0.0,0.0,0.0,0.478755,0.200373,0.195769,NaN,NaN


In [111]:
df['primary_topics'] = mf.idxmax(axis=0)

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [116]:
print(df.shape)
df.tail()

(28332, 26)


C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,primary_topics,sentiment
28327,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",...,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,I got 2 of these for my 8 yr old twins. My 11 ...,Xmas gift,Mom2twinsplus1,http://reviews.bestbuy.com/3545/5620410/review...,NaN,positive
28328,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",...,NaN,0.0,4,http://reviews.bestbuy.com/3545/5620410/review...,I bought this for my niece for a Christmas gif...,yes it is a great tablet.,fireman21,http://reviews.bestbuy.com/3545/5620410/review...,NaN,positive
28329,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",...,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,"Very nice for light internet browsing, keeping...",You get a lot for the price!,suzannalicious,http://reviews.bestbuy.com/3545/5620410/review...,NaN,positive
28330,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",...,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,This Tablet does absolutely everything I want!...,You get the entire World for less than $100!,SandyJ,http://reviews.bestbuy.com/3545/5620410/review...,NaN,positive
28331,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",...,NaN,0.0,4,http://reviews.bestbuy.com/3545/5620410/review...,"At ninety dollars, the expectionations are low...",You get what your paying for,Hung,http://reviews.bestbuy.com/3545/5620410/review...,NaN,positive


In [113]:
#Hint: a 'negative' review rating if has a rating betweeen 1 & 3 based on the distribution of review ratings. A 'postive' review rating is a 4 or 5.
df['sentiment'] = ["positive" if reviews_rating >= 4 else "negative" for reviews_rating in df['reviews.rating']]
df['sentiment'].value_counts(normalize=True)

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


positive    0.901631
negative    0.098369
Name: sentiment, dtype: float64

In [121]:
df['name'].value_counts()

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AmazonBasics AAA Performance Alkaline Batteries (36 Count)                               8343
AmazonBasics AA Performance Alkaline Batteries (48 Count) - Packaging May Vary           3728
Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers        2443
All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Black    2370
Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case                   1676
                                                                                         ... 
Amazon Echo Show - Black                                                                    1
Echo Dot (Previous generation)                                                              1
AmazonBasics Silicone Hot Handle Cover/Holder - Red                                         1
AmazonBasics Nespresso Pod Storage Drawer - 50 Capsule Capacity                             1
Two Door Top Load Pet Kennel Travel Crate Dog Cat Pet Cage C

In [117]:
agg = pd.pivot_table(df, values=['reviews.text'], 
                     index=['categories'], 
                     columns=['sentiment'], aggfunc='count')

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [118]:
agg = agg.reset_index()

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [120]:
agg.columns = ['categories', 'negative', 'positive']
print(agg.shape)
agg.head()

(60, 3)


C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,categories,negative,positive
0,"AA,AAA,Electronics Features,Health,Electronics...",518.0,3210.0
1,"AA,AAA,Health,Electronics,Health & Household,C...",1162.0,7181.0
2,"Accessories,USB Cables,Computers & Accessories...",NaN,6.0
3,"Amazon Echo,Home Theater & Audio,MP3 MP4 Playe...",49.0,552.0
4,"Amazon Echo,Networking,Home & Tools,Smart Home...",NaN,2.0


In [129]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(150, 500))

sns.set_color_codes('pastel')
sns.barplot(x='negative', y='categories', data=agg, label="Negative", color="b")

sns.set_color_codes('muted')
sns.barplot(x='positive', y='categories', data=agg, label="Positive", color="b")

ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(xlim=(0,500), ylabel="", xlabel="Count of Reviews")
sns.despine(left=True, bottom=True)

C:\Users\johnt\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling